In [2]:
import praw
from kafka import KafkaProducer, KafkaConsumer
import json

reddit = praw.Reddit(client_id='yQ83iAt0TWpfgiW07sTkOw', client_secret='q9na4iAgw25R8yBgLIDiNVaMwSp7EQ', user_agent='gabrissk', check_for_async=False)

In [3]:
class Post:
    def __init__(self, id, title, creation_date, subject) -> None:
        self.id = id
        self.title = title
        self.creation_date = creation_date
        self.comments = []
        self.subject = subject
    def add_comment(self, comment):
        self.comments.append(comment)

In [4]:
class Comment:
    def __init__(self, id, text, creation_date) -> None:
        self.id = id
        self.text = text
        self.creation_date = creation_date

In [ ]:
def process_comment(comment, new_post):
    # print(comment.body)
    new_post['comments'].append(
        {
            "id": comment.id, 
            "text": comment.body, 
            "creation_date": comment.created_utc
        }
    )

    for reply in comment.replies:
        process_comment(reply, new_post)

def process_subreddit_comments(subject, posts:list):
    subreddit = reddit.subreddit(subject)
    for post in subreddit.new(limit=10):
        # print(post.title)
        # print(post.selftext)
        # print(post.created_utc)
        new_post = {
            "id": post.id, 
            "title": post.title, 
            "creation_date": post.created_utc,
            "subject": subject,
            "comments": []
        }
        post.comments.replace_more()
        for comment in post.comments:
            process_comment(comment, new_post)
                
        # send_post_to_kafka(new_post)
        posts.append(new_post)


def send_post_to_kafka(posts):
    print("Sending to kafka...")
    producer = KafkaProducer(bootstrap_servers='localhost:9092')
    # post_dict = vars(posts)
    posts
    producer.send("reddit-dataviz-comments", json.dumps(posts).encode('utf-8')).get(timeout=10)
    producer.flush()
    producer.close

subjects = ["powerbi", "tableau", "qlikview", "looker", "datastudio", "domo"]
posts = []

for subject in subjects:
    process_subreddit_comments(subject, posts)
    print("Finished reading from ", subject)

send_post_to_kafka(posts)


In [ ]:
producer = KafkaProducer(bootstrap_servers='localhost:29002')